In [1]:
### Imports for wedscraping
from bs4 import BeautifulSoup
import requests
from IPython.core.formatters import catch_format_error
import regex as re
import pandas as pd

In [2]:
### urls for the three rightmove locations to scrape
b_site = 'https://www.rightmove.co.uk'
## Newcastle
#url = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E984&radius=3.0&sortType=10&index=0&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='
## Middlesbrough
#url = https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E933&radius=5.0&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=
## Sunderland
url = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1295&radius=3.0&index=0&sortType=10&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='

In [3]:
### Testing the website response
response = requests.get(url)
response

<Response [200]>

Start of scraping

In [4]:
###Creating a list of numbers for the index to navigate between all of the pages on rightmove

numbers = range(0,984,24)

index_num = []
for num in numbers:
  index_num.append(num)

In [ ]:
### Getting the links to all properties on all pages

# Initiating an empty list
links = []

# Using the index_num list to loop through each of the 42 pages on the site
for i in index_num:
  site = f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1295&radius=3.0&index={i}&sortType=10&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='
 

  resp = requests.get(site)
  soup = BeautifulSoup(resp.text, 'html.parser')

  # Looping through each page link to retrieve each property link
  for link in soup.find_all('div', class_  ='propertyCard-details'):
    try:
      # Appending the links list with each property link
      links.append(link.a.get('href'))
    except Exception as e:
      print('')

  for link in links:
    if link == '/properties/58852044#/?channel=RES_BUY':
      continue
    else:
      # Creating the full hyperlink from the scraped one using the base link (2nd cell)
      homes =  [b_site + x for x in links]

In [ ]:
### loop to colect all data from the property pages (price, bedrooms, house type and address)

prices = []
address = []
htype = []
bedrooms = []

for link in homes:

    # The error handling for each link append is for the properties that haven't been 
    # typed intot he site correctly. These links wil be handled after.
    resp = requests.get(link)
    soup = BeautifulSoup(resp.text, 'html.parser')

    try:
      prices.append(soup.find('div',class_  = '_1gfnqJ3Vtd1z40MlC0MzXu').text)
    except Exception as e:
      print(link)
    
    try:
      address.append(soup.find('div', class_ = 'OojFk4MTxFDKIfqreGNt0').text)
    except Exception as e:
      print(link)
    
    try:
      htype.append(soup.find_all('div',class_  = '_3OGW_s5TH6aUqi4uHum5Gy')[0].text)
    except Exception as e:
      print(link)

    try:
      bedrooms.append(soup.find_all('div',class_  = '_3OGW_s5TH6aUqi4uHum5Gy')[1].text)
    except Exception as e:
      print(link)

In [ ]:
### Creating a dictionary with each of the lists scraped
dict = {'price':prices, 'address':address, 'house_type':htype, 'bedroom':bedrooms}

In [ ]:
### Creating a dataframe with the dictionary
df = pd.DataFrame(dict)

In [ ]:
### Saving the scraped df as a csv
df.to_csv('scraped_data.csv')